In [24]:
import geopandas as gpd
import pandas as pd
import json
import copy
import os

### Konfiguration

In [25]:
state = 'WI'

In [26]:
pos = {
    'IA': {
        'name': 'Iowa',
        'scale': 2530,
        'translate': [80, 310],
        'arrow': 2
    },
    'PA': {
        'name': 'Pennsylvania',
        'scale': 4000,
        'translate': [-800, 380],
        'arrow': 2
    },
    'WI': {
        'name': 'Wisconsin',
        'scale': 2500,
        'translate': [-20,410],
        'arrow': 2
    },
    'MI': {
        'name': 'Michigan',
        'scale': 2070,
        'translate': [-90,370],
        'arrow': 2
    }
}

### Daten runterfiltern

In [27]:
d1 = gpd.GeoDataFrame(pd.read_pickle('temp/windofchange.pkl'))
d1

,fips,county,state,geometry,winner,shifted,population,_merge,datensatz
0,13027,Brooks,GA,"MULTIPOLYGON (((-83.73616 31.03768, -83.57396 ...",Trump,24.0,16245.0,both,NaN
1,31095,Jefferson,NE,"MULTIPOLYGON (((-97.36869 40.35039, -96.91606 ...",Trump,19.0,7054.0,both,NaN
2,31095,Jefferson,NE,"MULTIPOLYGON (((-97.36869 40.35039, -96.91606 ...",Trump,-9.0,7054.0,both,NaN
3,51683,Manassas,VA,"MULTIPOLYGON (((-77.52666 38.73320, -77.49545 ...",Trump,-6.0,42696.0,both,NaN
4,56021,Laramie,WY,"MULTIPOLYGON (((-105.27824 41.65666, -104.6533...",Trump,3.0,100984.0,both,NaN
...,...,...,...,...,...,...,...,...,...
3338,9007,NaN,NaN,None,Trump,16.0,NaN,right_only,Nikolai
3339,9008,NaN,NaN,None,Harris,0.0,NaN,right_only,Nikolai
3340,9009,NaN,NaN,None,Trump,0.0,NaN,right_only,Nikolai
3341,9010,NaN,NaN,None,Trump,2.0,NaN,right_only,Nikolai


In [28]:
d2 = d1[d1.state == state]
d2

,fips,county,state,geometry,winner,shifted,population,_merge,datensatz
31,55113,Sawyer,WI,"MULTIPOLYGON (((-91.55128 46.15704, -90.92458 ...",Harris,5.0,18552.0,both,NaN
164,55087,Outagamie,WI,"MULTIPOLYGON (((-88.73680 44.59060, -88.60608 ...",Trump,1.0,193234.0,both,NaN
278,55125,Vilas,WI,"MULTIPOLYGON (((-90.04396 45.98195, -89.92884 ...",Harris,8.0,23885.0,both,NaN
440,55015,Calumet,WI,"MULTIPOLYGON (((-88.40351 44.00129, -88.40407 ...",Harris,-18.0,53199.0,both,NaN
490,55017,Chippewa,WI,"MULTIPOLYGON (((-91.66565 45.20799, -91.54223 ...",Harris,-8.0,66970.0,both,NaN
...,...,...,...,...,...,...,...,...,...
2910,55063,La Crosse,WI,"MULTIPOLYGON (((-91.42357 43.98430, -91.32214 ...",Trump,-2.0,120486.0,both,NaN
3076,55025,Dane,WI,"MULTIPOLYGON (((-89.83813 43.20606, -89.73174 ...",NaN,20.0,575347.0,both,NaN
3108,55101,Racine,WI,"MULTIPOLYGON (((-88.30638 42.84210, -88.06992 ...",Trump,-9.0,196613.0,both,NaN
3162,55095,Polk,WI,"MULTIPOLYGON (((-92.88670 45.64415, -92.52911 ...",Trump,-10.0,45762.0,both,NaN


In [29]:
d2.to_file('temp/'+state+'.geojson', driver='GeoJSON')

In [30]:
!geo2topo temp/{state}.geojson>temp/{state}.topo.json

### Arrowmap

In [31]:
with open('temp/'+state+'.topo.json', 'r') as f:
    stateData = json.load(f)

In [32]:
with open('specs/arrow-mw.vg.json', 'r') as f:
    spec_mw = json.load(f)

In [33]:
spec_mw['data'][1]['values'] = stateData
spec_mw['data'][1]['format']['feature'] = state
spec_mw['data'][0]['transform'] = [
        {
          "type": "filter",
          "expr": "datum.properties.name === '"+pos[state]['name']+"'"
        }
      ]
spec_mw['projections'][0]['scale'] = pos[state]['scale']
spec_mw['projections'][0]['translate'] = pos[state]['translate']
spec_mw['signals'][0]['value'] = pos[state]['arrow']
spec_mw

{'$schema': 'https://vega.github.io/schema/vega/v5.json',
 'description': 'A choropleth map depicting U.S. unemployment rates by county in 2009.',
 'width': 350,
 'height': 250,
 'autosize': 'none',
 'signals': [{'name': 'len_factor', 'value': 2},
  {'name': 'stroke_factor', 'value': 1},
  {'name': 'county_borders', 'value': 0.5}],
 'projections': [{'name': 'projection',
   'type': 'albersUsa',
   'scale': 2500,
   'translate': [-20, 410]}],
 'scales': [{'name': 'color',
   'type': 'quantize',
   'domain': [-56, 56],
   'reverse': True,
   'range': ['#ce6661',
    '#df7e76',
    '#ef978c',
    '#99b4d7',
    '#7ea1cc',
    '#618ec0']},
  {'name': 'opacity',
   'type': 'linear',
   'domain': [0, 5.570860980419826, 11.141721960839652, 56],
   'range': [0, 0.666666, 1, 1]},
  {'name': 'size',
   'type': 'linear',
   'domain': [0, 55],
   'range': [10, {'signal': 'len_factor*len_factor*300'}]},
  {'name': 'population',
   'type': 'linear',
   'domain': {'data': 'counties', 'field': 'proper

In [34]:
folder_path = 'temp/specs/'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [35]:
with open('temp/specs/arrow-'+state+'-mw.vg.json', 'w') as f:
    json.dump(spec_mw, f, indent=4)

### Bubblemap

In [36]:
with open('specs/bubble-mw.vg.json', 'r') as f:
    spec_mw = json.load(f)

In [38]:
spec_mw['data'][1]['values'] = stateData
spec_mw['data'][1]['format']['feature'] = state
spec_mw['data'][0]['transform'] = [
        {
          "type": "filter",
          "expr": "datum.properties.name === '"+pos[state]['name']+"'"
        }
      ]
spec_mw['projections'][0]['scale'] = pos[state]['scale']
spec_mw['projections'][0]['translate'] = pos[state]['translate']

In [39]:
with open('temp/specs/bubble-'+state+'-mw.vg.json', 'w') as f:
    json.dump(spec_mw, f, indent=4)

In [40]:
!vg2svg temp/specs/arrow-{state}-mw.vg.json svgs/arrow-{state}-mw.svg
!vg2svg temp/specs/bubble-{state}-mw.vg.json svgs/bubble-{state}-mw.svg

(node:59584) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)
(node:59587) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)
